# 2. Using deep neural networks for regression
## 2.1. How to plan a machine learning problem
* Khi xây dựng một deep neural network, hãy theo dõi các bước sau:
  * Vạch ra vấn đề ta đang muốn giải quyết.
  * Xác định input và output của model.
  * Xác định cost function.
  * Tạo một network.
  * Đào tạo và hiệu chỉnh network.
  
## 2.2. Defining our example problem
* Chúng ta sẽ sử dụng dataset ***wine quality*** [chất lượng rượu] [tại đây](https://archive.ics.uci.edu/ml/datasets/wine+quality) cho các bài toán cho đến khi tôi thông báo ta sẽ sử dụng một dataset mới.
* Có tổng số 4898 observation trong dataset này. Con số này có vẻ khá lơn với bài toán regression cổ điển nhưng lại khá nhỏ cho deep neural network.
* Có 10 đặc điểm hóa học mà ta cần dùng để dự đoán cho target variable (`alcohol`) và chúng đều là các continous variable. Vùng blue square là independent variable và red square là dependent variable:<br>
  ![](./images/02.00.png)

## 2.3. Loading the dataset
* Chúng ta cần load dữ liệu wine quality lên như sau:

In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
TRAIN_DATA = "./data/train/train_data.csv"
VAL_DATA = "./data/val/val_data.csv"
TEST_DATA = "./data/test/test_data.csv"

In [3]:
def load_data():
    '''Load train, val and test datasets from disk.'''
    train = pd.read_csv(TRAIN_DATA)
    val = pd.read_csv(VAL_DATA)
    test = pd.read_csv(TEST_DATA)
    
    '''Using sklearn's StandardScaler to scale our data to 0 mean and unit variance.'''
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    val = scaler.transform(val)
    test = scaler.transform(test)
    
    '''We will use a dict to keep all this data.'''
    data = dict()
    data['train_y'] = train[:, 10]
    data['train_X'] = train[:, 0:9]
    data['val_y'] = val[:, 10]
    data['val_X'] = val[:, 0:9]
    data['test_y'] = test[:, 10]
    data['test_X'] = test[:, 0:9]
    
    '''Keep the `scaler` so we can unscale prediction in the future.'''
    data['scaler'] = scaler
    
    return data

## 2.4. Defining our cost function
* Đối với các bài toán hồi quy, cost function hay được dùng là **Root Mean Squared Error - RMSE** và **Mean Absolute Error - MAE**. Ở đây chúng ta sẽ dùng MAE, công thức như sau:
  $$\mathrm{MAE} = \dfrac{1}{n} \sum_{j = 1}^n |y_j - \widehat{y_j}|$$

* Bạn cũng có thể sử dụng RMSE như sau:
  $$\mathrm{RMSE} = \sqrt{\dfrac{1}{n} \sum_{j = 1}^n (y_j - \widehat{y_j})^2}$$

* Có thể bạn sẽ bối rối ụa hai cái này làm sao biết nên chọn cái nào:
  * Trong trường hợp lỗi phân bố đều trên training data thì $\mathrm{RMSE} = \mathrm{MAE}$, ví dụ ta có 10 data point và 10 data point này đều có $|y_j - \widehat{y_j}| = 5$.
  * Còn nếu training data có outlier thì $\mathrm{RMSE}$ sẽ **lớn hơn rất nhiều** so với $\mathrm{MAE}$.
* Không có một chuẩn nào đặt ra cho việc lựa chọn cost function, chủ yếu là do cảm nhận của bạn vào dataset. Nhưng về diễn giải, $\mathrm{MAE}$ dễ hiểu hơn so với $\mathrm{RMSE}$.

# 3. Building and MLP in Keras
* Một model của Keras là một tập hợp của các layer và chúng ta cần định nghĩa tập các layer này cho Keras hiểu.
* Keras hiện có hai API để đào tạo model. Trong ví dụ này, chúng ta sẽ sử dụng **Functional API** - nó khá là dài dòng về mặt coding nhưng khả năng tùy biến của nó cao. Hầu hết các pro đều sài Functional API.

* Model MLP của chúng ta sẽ cần:
  * Một input layer.
  * Một hidden layer.
  * Một output layer.

## 3.1. Input layer shape
* Input của chúng ta là một matrix với số dòng là số lượng observation và số cột là số lượng feature của dataset. Vậy input matrix của chúng ta có shape là $\text{số lượng observation} \times 10$.
* Tuy nhiên, chúng ta không cần phải xác định chính xác shape của input matrix. TensorFlow và Keras cho phép chúng ta định nghĩa giá trị `None` cho biến **placeholder** và chúng ta hoàn toàn có thể định nghĩa lại biến placeholder này về sau.

## 3.2. Hidden layer shape
* Hidden layer của chúng ta sẽ có 32 neutron. Tại thời điểm này chúng ta không thể biết chính xác chúng ta cần bao nhiêu neutron cho network vì đây là một hyperparameter và ta cần hiệu chỉnh nó về sau. Việc xác định kiến trúc của một network là một vấn đề mở rộng trong deep learning.
* Vì chúng ta có 32 neutron trong hidden layer, input layer của chúng ta gồm 10 feature nên shape của hidden layer là (10, 32).

## 3.3. Output layer shape
* Output layer của chúng ta sẽ bao gồm duy nhất một neuron, nó nhận vào 32 neutron của hidden layer như là input của nó và tiến hành dự đoán ra một giá trị $\widehat{y}$ duy nhất cho từng data point.
* Vậy model MLP của chúng ta sẽ trông thế này:<br>
  <center>

    ![](./images/02.01.png)

  </center>
  
## 3.4. Neural network architecture
* Bây giờ chúng ta sẽ định nghĩa input và output.

In [4]:
from keras.layers import Input, Dense
from keras.models import Model

In [8]:
def build_network(input_features=None):
    inputs = Input(shape=(input_features,), name='input') # input layer
    x = Dense(32, activation='relu', name='hidden')(inputs) # hidden layer với 32 neuron
    prediction = Dense(1, activation='linear', name='final')(x) # output layer với 1 neuron
    
    model = Model(inputs=inputs, outputs=prediction) # build model với input và output
    model.compile(optimizer='adam', loss='mean_absolute_error') # biên dịch model với adam optimizer và loss mà MAE
    
    return model

* Hãy khám phá hàm trên:
  * Dòng code số 4 ta dùng activation function là `linear`, điều này cũng giống như việc chúng ta không sử dụng bất kì activation function nào, đây là những gì mà chúng ta muốn cho bài toán hồi quy
  * Dòng code số 6, ta định nghĩa đâu là input layer và đâu là output layer cho `Model` object.
  * Dòng code số 7, ta định nghĩa Adam optimizer cho `optimizer` và MAE cho `loss` function.
* Bây giờ chúng ta có thể gọi hàm `build_model()` để xây dựng một neural network như sau:

In [9]:
model = build_network(input_features=10)

* Giả sử bây giờ ta muốn hiệu chỉnh các hyperparameter của Adam optimizer thì ta có thể làm như sau:

In [12]:
from keras.optimizers import adam_v2

def build_network_with_adam_hyperparams(input_features=None):
    inputs = Input(shape=(input_features,), name='input') # input layer
    x = Dense(32, activation='relu', name='hidden')(inputs) # hidden layer với 32 neuron
    prediction = Dense(1, activation='linear', name='final')(x) # output layer với 1 neuron
    
    adam_optimizer = adam_v2.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
    
    model = Model(inputs=inputs, outputs=prediction) # build model với input và output
    model.compile(optimizer=adam_optimizer, loss='mean_absolute_error') # biên dịch model với adam optimizer và loss mà MAE
    
    return model

## 3.5. Training the Keras model
